### Demo Preparation Notebook

**Please Note**: This notebook and demo are NOT intended to be used as learning materials. To gain
a thorough understanding of the DataJoint workflow for calcium imaging, please
see our [`tutorial`](./tutorial.ipynb) notebook.

In [1]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

import datajoint as dj
import datetime
import numpy as np
from workflow_calcium_imaging.pipeline import subject, session, scan, imaging
from element_calcium_imaging import imaging_report
import suite2p

[2023-03-07 17:53:28,952][INFO]: Connecting root@fakeservices.datajoint.io:3306
[2023-03-07 17:53:28,959][INFO]: Connected root@fakeservices.datajoint.io:3306


In [2]:
subject.Subject.insert1(
    dict(
        subject='subject1',
        subject_birth_date='2023-01-01',
        sex='U',
    )
)

In [ ]:
Equipment.insert1(dict(scanner="Mesoscope"))

In [3]:
session_key = dict(subject='subject1', 
                   session_datetime=datetime.datetime.now())

session.Session.insert1(session_key)

session.SessionDirectory.insert1(
    dict(
        session_key, 
        session_dir='subject1/session1'
    )
)

In [4]:
scan.Scan.insert1(
    dict(
        session_key,
        scan_id=0,
        scanner="Mesoscope",
        acq_software='ScanImage',
    )
)

In [ ]:
populate_settings = {"display_progress": True}

scan.ScanInfo.populate(**populate_settings)

In [5]:
suite2p_params = suite2p.default_ops()
suite2p_params['nonrigid']=False

imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    paramset_idx=0,
    params=suite2p_params,
    paramset_desc='Default parameter set for suite2p'
)

In [6]:
imaging.ProcessingTask.insert1(
    dict(
        session_key,
        scan_id=0,
        paramset_idx=0,
        task_mode='load', # load or trigger
        processing_output_dir='subject1/session1/suite2p',
    )
)

imaging.Processing.populate(**populate_settings)

In [7]:
imaging.Curation.insert1(
    dict(
        session_key,
        scan_id=0,
        paramset_idx=0,
        curation_id=0,
        curation_time=datetime.datetime.now(),
        curation_output_dir='subject1/session1/suite2p',
        manual_curation=False,
    )
)

imaging.MotionCorrection.populate(**populate_settings)
imaging.Segmentation.populate(**populate_settings)
imaging.Fluorescence.populate(**populate_settings)
imaging.Activity.populate(**populate_settings)
imaging_report.ScanLevelReport.populate()
imaging_report.TraceReport.populate()

Processing:   0%|          | 0/1 [00:00<?, ?it/s]

{'data_path': ['/workspaces/workflow-calcium-imaging/example_data/subject1/session1'], 'tiff_list': ['/workspaces/workflow-calcium-imaging/example_data/subject1/session1/example1.tif']}
FOUND BINARIES AND OPS IN ['/workspaces/workflow-calcium-imaging/example_data/subject1/session1/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
binary path: /workspaces/workflow-calcium-imaging/example_data/subject1/session1/suite2p/plane0/data.bin
NOTE: Applying builtin classifier at /usr/local/lib/python3.7/site-packages/suite2p/classifiers/classifier.npy
----------- ROI DETECTION
Binning movie in chunks of length 29
Binned movie of size [103,504,506] created in 2.29 sec.
ROIs: 200, cost: 0.2719, time: 37.3356
ROIs: 400, cost: 0.2142, time: 63.2455
ROIs: 547, cost: 0.1907, time: 84.7919
ROIs: 583, cost: 0.1853, time: 98.0249
ROIs: 599, cost: 0.1835, time: 109.7812
ROIs: 599, cost: 0.4262, time: 114.3391
ROIs: 599, c

Activity: 100%|██████████| 1/1 [00:00<00:00,  3.69it/s]
